# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 3 2022 11:24AM,243373,10,MSP212729,"Methapharm, Inc.",Released
1,Aug 3 2022 11:24AM,243373,10,MSP212767,"Methapharm, Inc.",Released
2,Aug 3 2022 11:24AM,243373,10,MSP212768,"Methapharm, Inc.",Released
3,Aug 3 2022 11:24AM,243373,10,MSP212769,"Methapharm, Inc.",Released
4,Aug 3 2022 11:24AM,243373,10,MSP212770,"Methapharm, Inc.",Released
5,Aug 3 2022 11:24AM,243373,10,MSP212771,"Methapharm, Inc.",Released
6,Aug 3 2022 11:24AM,243373,10,MSP212773,"Methapharm, Inc.",Released
7,Aug 3 2022 11:24AM,243373,10,MSP212774,"Methapharm, Inc.",Released
8,Aug 3 2022 11:24AM,243373,10,MSP212772,"Methapharm, Inc.",Released
9,Aug 3 2022 11:24AM,243373,10,MSP212776,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,243366,Released,22
47,243367,Released,1
48,243370,Released,17
49,243373,Released,26
50,243374,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243366,NaN,NaN,22.0
243367,NaN,NaN,1.0
243370,NaN,NaN,17.0
243373,NaN,NaN,26.0
243374,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243251,0.0,3.0,21.0
243270,0.0,0.0,1.0
243274,17.0,8.0,2.0
243294,0.0,0.0,2.0
243304,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243251,0,3,21
243270,0,0,1
243274,17,8,2
243294,0,0,2
243304,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243251,0,3,21
1,243270,0,0,1
2,243274,17,8,2
3,243294,0,0,2
4,243304,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243251,,3,21
1,243270,,,1
2,243274,17,8,2
3,243294,,,2
4,243304,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 3 2022 11:24AM,243373,10,"Methapharm, Inc."
26,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc."
32,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.
49,Aug 3 2022 11:19AM,243366,10,ISDIN Corporation
71,Aug 3 2022 11:17AM,243367,12,LF of America Corp.
72,Aug 3 2022 11:16AM,243365,10,Bio-PRF
73,Aug 3 2022 11:13AM,243364,20,"ACI Healthcare USA, Inc."
78,Aug 3 2022 11:12AM,243360,10,Eywa Pharma Inc.
79,Aug 3 2022 11:06AM,243362,12,"SD Head USA, LLC"
80,Aug 3 2022 11:04AM,243361,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 3 2022 11:24AM,243373,10,"Methapharm, Inc.",,,26
1,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc.",,,6
2,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.,,,17
3,Aug 3 2022 11:19AM,243366,10,ISDIN Corporation,,,22
4,Aug 3 2022 11:17AM,243367,12,LF of America Corp.,,,1
5,Aug 3 2022 11:16AM,243365,10,Bio-PRF,,,1
6,Aug 3 2022 11:13AM,243364,20,"ACI Healthcare USA, Inc.",,,5
7,Aug 3 2022 11:12AM,243360,10,Eywa Pharma Inc.,,,1
8,Aug 3 2022 11:06AM,243362,12,"SD Head USA, LLC",,,1
9,Aug 3 2022 11:04AM,243361,16,TASA USA Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 11:24AM,243373,10,"Methapharm, Inc.",26,,
1,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc.",6,,
2,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.,17,,
3,Aug 3 2022 11:19AM,243366,10,ISDIN Corporation,22,,
4,Aug 3 2022 11:17AM,243367,12,LF of America Corp.,1,,
5,Aug 3 2022 11:16AM,243365,10,Bio-PRF,1,,
6,Aug 3 2022 11:13AM,243364,20,"ACI Healthcare USA, Inc.",5,,
7,Aug 3 2022 11:12AM,243360,10,Eywa Pharma Inc.,1,,
8,Aug 3 2022 11:06AM,243362,12,"SD Head USA, LLC",1,,
9,Aug 3 2022 11:04AM,243361,16,TASA USA Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 11:24AM,243373,10,"Methapharm, Inc.",26,,
1,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc.",6,,
2,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.,17,,
3,Aug 3 2022 11:19AM,243366,10,ISDIN Corporation,22,,
4,Aug 3 2022 11:17AM,243367,12,LF of America Corp.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 11:24AM,243373,10,"Methapharm, Inc.",26.0,NaN,NaN
1,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc.",6.0,NaN,NaN
2,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.,17.0,NaN,NaN
3,Aug 3 2022 11:19AM,243366,10,ISDIN Corporation,22.0,NaN,NaN
4,Aug 3 2022 11:17AM,243367,12,LF of America Corp.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 11:24AM,243373,10,"Methapharm, Inc.",26.0,0.0,0.0
1,Aug 3 2022 11:24AM,243374,10,"Methapharm, Inc.",6.0,0.0,0.0
2,Aug 3 2022 11:21AM,243370,20,Alumier Labs Inc.,17.0,0.0,0.0
3,Aug 3 2022 11:19AM,243366,10,ISDIN Corporation,22.0,0.0,0.0
4,Aug 3 2022 11:17AM,243367,12,LF of America Corp.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5596617,196.0,8.0,17.0
12,486729,2.0,0.0,0.0
15,1459833,49.0,3.0,0.0
16,486715,2.0,0.0,0.0
18,243325,1.0,0.0,0.0
19,1216711,16.0,1.0,0.0
20,486734,22.0,0.0,0.0
21,1216598,5.0,0.0,0.0
22,486688,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5596617,196.0,8.0,17.0
1,12,486729,2.0,0.0,0.0
2,15,1459833,49.0,3.0,0.0
3,16,486715,2.0,0.0,0.0
4,18,243325,1.0,0.0,0.0
5,19,1216711,16.0,1.0,0.0
6,20,486734,22.0,0.0,0.0
7,21,1216598,5.0,0.0,0.0
8,22,486688,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,196.0,8.0,17.0
1,12,2.0,0.0,0.0
2,15,49.0,3.0,0.0
3,16,2.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,16.0,1.0,0.0
6,20,22.0,0.0,0.0
7,21,5.0,0.0,0.0
8,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,196.0
1,12,Released,2.0
2,15,Released,49.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,8.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0
Released,196.0,2.0,49.0,2.0,1.0,16.0,22.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0
2,Released,196.0,2.0,49.0,2.0,1.0,16.0,22.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0
2,Released,196.0,2.0,49.0,2.0,1.0,16.0,22.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()